In [1]:
import numpy as np
import scipy as sp
import matplotlib as plt
import importlib
from tqdm import tqdm

# Dynamically import modules
amp = importlib.import_module("amp")
pca_pack = importlib.import_module("pca_pack")
preprocessing = importlib.import_module("preprocessing")
emp_bayes = importlib.import_module("emp_bayes")
hierarchical = importlib.import_module("hierarchical_clustering_modalities")
pipeline = importlib.import_module("complete_pipeline")

# Reload to reflect any changes made without restarting kernel
importlib.reload(amp)
importlib.reload(pca_pack)
importlib.reload(preprocessing)
importlib.reload(emp_bayes)
importlib.reload(hierarchical)
importlib.reload(pipeline)

# Now access objects from reloaded modules
ebamp_multimodal = amp.ebamp_multimodal
MultiModalityPCA = pca_pack.MultiModalityPCA
MultiModalityPCADiagnostics = preprocessing.MultiModalityPCADiagnostics
ClusterEmpiricalBayes = emp_bayes.ClusterEmpiricalBayes
ModalityClusterer = hierarchical.ModalityClusterer
MultimodalPCAPipeline = pipeline.MultimodalPCAPipelineSimulation
MultimodalPCAPipelineClustering = pipeline.MultimodalPCAPipelineClusteringSimulation

In [2]:
# --- Global Parameters ---
p_list = [1000, 1000, 1500]  # Feature dimensions for each modality
r_list = [1, 2, 1]           # Rank per modality (number of PCs)

In [3]:
# --- Utility: Rademacher generator ---
def generate_rademacher(shape):
    return np.random.choice([-1, 1], size=shape)

In [4]:
# --- Utility: Reconstruction error ---
def reconstruction_error(U_est, U_true):
    P_est = U_est @ U_est.T
    P_true = U_true @ U_true.T
    return np.linalg.norm(P_est - P_true, 'fro')**2 / (U_true.shape[0]**2)

In [5]:
# --- Main experiment runner ---
def run_amp_comparison_experiment(n=2000, num_trials=100, amp_iters=15):
    errors_clustered = {i: [] for i in range(3)}
    errors_same = {i: [] for i in range(3)}
    errors_distinct = {i: [] for i in range(3)}

    for trial in tqdm(range(num_trials), desc="Trials"):
        # Generate U
        U1 = generate_rademacher((n, r_list[0]))
        U2 = np.hstack([U1[:, :r_list[0]], generate_rademacher((n, r_list[1] - r_list[0]))])
        U3 = generate_rademacher((n, r_list[2]))
        U_true = [U1, U2, U3]

        # Generate V
        V1 = generate_rademacher((p_list[0], r_list[0]))
        V2 = generate_rademacher((p_list[1], r_list[1]))
        V3 = generate_rademacher((p_list[2], r_list[2]))

        # D matrices
        D1 = np.diag([5 * (i+1) for i in range(r_list[0])])
        D2 = np.diag([5 * (i+1) for i in range(r_list[1])])
        D3 = np.diag([5 * (i+1) for i in range(r_list[2])])

        # Noise
        Z1 = np.random.randn(n, p_list[0]) / np.sqrt(n)
        Z2 = np.random.randn(n, p_list[1]) / np.sqrt(n)
        Z3 = np.random.randn(n, p_list[2]) / np.sqrt(n)

        # Observed data
        X1 = (1/n) * U1 @ D1 @ V1.T + Z1
        X2 = (1/n) * U2 @ D2 @ V2.T + Z2
        X3 = (1/n) * U3 @ D3 @ V3.T + Z3
        X_list = [X1, X2, X3]
        K_list = r_list

        # AMP with clustering
        pipe_cluster = MultimodalPCAPipelineClustering()
        result_cluster = pipe_cluster.denoise_amp(X_list, K_list, compute_clusters=True, num_clusters=2, amp_iters=amp_iters)
        U_cluster = result_cluster["U_denoised"]

        # AMP with same-cluster (all modalities in one group)
        pipe_same = MultimodalPCAPipeline()
        result_same = pipe_same.denoise_amp(X_list, K_list, cluster_labels_U=np.array([0, 0, 0]), amp_iters=amp_iters)
        U_same = result_same["U_denoised"]

        # AMP with different clusters (1 per modality)
        pipe_distinct = MultimodalPCAPipeline()
        result_distinct = pipe_distinct.denoise_amp(X_list, K_list, cluster_labels_U=np.array([0, 1, 2]), amp_iters=amp_iters)
        U_distinct = result_distinct["U_denoised"]

        # Compute reconstruction errors
        for i in range(3):
            errors_clustered[i].append(reconstruction_error(U_cluster[i][:, :, -1], U_true[i]))
            errors_same[i].append(reconstruction_error(U_same[i][:, :, -1], U_true[i]))
            errors_distinct[i].append(reconstruction_error(U_distinct[i][:, :, -1], U_true[i]))

        print(
            f"[Trial {trial+1}] "
            f"Reconstruction Errors - "
            f"Clustering: {[round(errors_clustered[i][-1], 5) for i in range(3)]}, "
            f"Same Cluster: {[round(errors_same[i][-1], 5) for i in range(3)]}, "
            f"Distinct Clusters: {[round(errors_distinct[i][-1], 5) for i in range(3)]}"
        )

    return errors_clustered, errors_same, errors_distinct

In [6]:
errors_clustered, errors_same, errors_distinct = run_amp_comparison_experiment(n=5000, num_trials=50, amp_iters=10)

Trials:   2%|▏         | 1/50 [00:42<34:49, 42.63s/it]

[Trial 1] Reconstruction Errors - Clustering: [np.float64(0.01739), np.float64(0.03232), np.float64(0.02177)], Same Cluster: [np.float64(0.06761), np.float64(0.0678), np.float64(0.07753)], Distinct Clusters: [np.float64(0.07491), np.float64(0.0779), np.float64(0.02164)]


Trials:   2%|▏         | 1/50 [00:50<41:30, 50.82s/it]


KeyboardInterrupt: 